In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Preparing the test data 

#Load the test data
test_data = pd.read_csv("C:/Users/HP/Downloads/CERTIFICATION/archive/Google_Stock_Price_Test.csv")

In [3]:
# Convert Date column to datetime format
test_data['Date'] = pd.to_datetime(test_data['Date'])

In [4]:
# Sort data by Date (if not already sorted)
test_data = test_data.sort_values('Date')

In [5]:
# Select relevant features (you may adjust this based on your requirements)
features = ['Open', 'High', 'Low', 'Close', 'Volume']
test_data = test_data[features]

In [6]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
test_data_scaled = scaler.fit_transform(test_data)

In [7]:
# Create sequences
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequence = data[i : (i + sequence_length)]
        sequences.append(sequence)
    return sequences

sequence_length = 10  # Adjust as needed
test_sequences = create_sequences(test_data_scaled, sequence_length)

In [8]:
# Convert to numpy array
test_sequences = np.array(test_sequences)

In [9]:
# Split sequences into features (X) and target (y)
X_test = test_sequences[:, :-1]
y_test = test_sequences[:, -1]

In [10]:
# Reshape input data to be 3D (samples, timesteps, features)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], len(features))

In [11]:
# Now X_test and y_test can be used to evaluate the LSTM model

In [12]:
#Preparing the train data 

In [13]:
# Load the CSV file into a DataFrame
df = pd.read_csv("C:/Users/HP/Downloads/CERTIFICATION/archive/Google_Stock_Price_Train.csv")

In [14]:
# Convert the date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

C:\Users\HP\AppData\Local\Temp\ipykernel_17116\4210372300.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [15]:
# Convert the date column back to a string with a consistent format (e.g., YYYY-MM-DD)
df['Date'] = df['Date'].dt.strftime('%m-%d-%Y')

# Save the DataFrame back to a CSV file
df.to_csv("updated_file.csv", index=False)

In [16]:
# Load your training data
train_data = pd.read_csv("C:/Users/HP/Downloads/CERTIFICATION/archive/Google_Stock_Price_Test.csv")

In [17]:
# Convert Date column to datetime format
train_data['Date'] = pd.to_datetime(train_data['Date'])

In [18]:
# Sort data by Date (if not already sorted)
train_data = train_data.sort_values('Date')

In [19]:
# Select relevant features
features = ['Open', 'High', 'Low', 'Close', 'Volume']
train_data = train_data[features]

In [20]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(train_data)


In [21]:
# Create sequences
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequence = data[i : (i + sequence_length)]
        sequences.append(sequence)
    return sequences

sequence_length = 10  # Adjust as needed
train_sequences = create_sequences(train_data_scaled, sequence_length)

In [22]:
# Convert to numpy array
train_sequences = np.array(train_sequences)

In [23]:
# Split sequences into features (X) and target (y)
X_train = train_sequences[:, :-1]
y_train = train_sequences[:, -1]

In [24]:
# Reshape input data to be 3D (samples, timesteps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], len(features))

# Now X_train and y_train can be used to train the LSTM model


In [25]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten
import numpy as np

In [26]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=False))  # Ensure return_sequences=False
model.add(Flatten())  # Add a Flatten layer to convert output to 2D
model.add(Dense(units=1))

In [27]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [28]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10

1/1 [==============================] - 6s 6s/step - loss: 0.3304
Epoch 2/10
1/1 [==============================] - 0s 20ms/step - loss: 0.2642
Epoch 3/10
1/1 [==============================] - 0s 20ms/step - loss: 0.2049
Epoch 4/10
1/1 [==============================] - 0s 17ms/step - loss: 0.1531
Epoch 5/10
1/1 [==============================] - 0s 15ms/step - loss: 0.1098
Epoch 6/10
1/1 [==============================] - 0s 20ms/step - loss: 0.0770
Epoch 7/10
1/1 [==============================] - 0s 18ms/step - loss: 0.0573
Epoch 8/10
1/1 [==============================] - 0s 16ms/step - loss: 0.0531
Epoch 9/10
1/1 [==============================] - 0s 14ms/step - loss: 0.0635
Epoch 10/10
1/1 [==============================] - 0s 15ms/step - loss: 0.0795


In [29]:
# Predictions
predictions = model.predict(X_test)

1/1 [==============================] - 1s 1s/step


In [30]:
# Reshape predictions array to match the shape of y_test_actual
predictions = predictions.reshape(-1, len(features))

In [34]:
# Ensure that both arrays have the same shape and number of samples
min_samples = min(len(predictions), len(y_test_actual))
predictions = predictions[:min_samples]
y_test_actual = y_test_actual[:min_samples]

In [35]:
# Calculate the mean squared error
mse = mean_squared_error(y_test_actual, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 573875443329.2654
